In [67]:
text = ("General intelligence (the ability to solve an arbitrary problem) "
        "is among the field's long-term goals. To solve these problems, AI researchers "
        "have adapted and integrated a wide range of problem-solving techniques, including "
        "search and mathematical optimization, formal logic, artificial neural networks, "
        "and methods based on statistics, probability, and economics")

with open("example2.txt", "w") as f:
    f.write(text)

In [68]:
data = open("example2.txt", "r").read()
chars = list(set(data))
X_size = len(chars)
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}

In [69]:
import numpy as np
H_size = 10
T_steps = 25
learning_rate = 1e-1

Wf = np.random.randn(H_size, H_size + X_size) * 0.1
Wi = np.random.randn(H_size, H_size + X_size) * 0.1
Wo = np.random.randn(H_size, H_size + X_size) * 0.1
Wc = np.random.randn(H_size, H_size + X_size) * 0.1
Wy = np.random.randn(X_size, H_size) * 0.1

bf = np.zeros((H_size, 1))
bi = np.zeros((H_size, 1))
bo = np.zeros((H_size, 1))
bc = np.zeros((H_size, 1))
by = np.zeros((X_size, 1))

In [70]:
def sigmoid(x): return 1 / (1 + np.exp(-x))
def dsigmoid(y): return y * (1 - y)
def tanh(x): return np.tanh(x)
def dtanh(y): return 1 - y ** 2

def softmax(v):
    e = np.exp(v - np.max(v))
    return e / np.sum(e)

In [71]:
def sample(h, C, seed_idx, n):
    x = np.zeros((X_size, 1))
    x[seed_idx] = 1
    indices = []
    for t in range(n):
        z = np.vstack((h, x))
        f = sigmoid(Wf @ z + bf)
        i = sigmoid(Wi @ z + bi)
        o = sigmoid(Wo @ z + bo)
        C_bar = tanh(Wc @ z + bc)
        C = f * C + i * C_bar
        h = o * tanh(C)
        y = Wy @ h + by
        p = softmax(y)
        idx = np.random.choice(range(X_size), p=p.ravel())
        x = np.zeros((X_size, 1))
        x[idx] = 1
        indices.append(idx)
    return ''.join(idx_to_char[i] for i in indices)

In [72]:
h = np.zeros((H_size, 1))
C = np.zeros((H_size, 1))
smooth_loss = -np.log(1.0 / X_size) * T_steps

for iteration in range(1000):
    if iteration * T_steps + T_steps + 1 >= len(data):
        h = np.zeros((H_size, 1))
        C = np.zeros((H_size, 1))
        continue

    inputs = [char_to_idx[ch] for ch in data[iteration*T_steps : iteration*T_steps+T_steps]]
    targets = [char_to_idx[ch] for ch in data[iteration*T_steps+1 : iteration*T_steps+T_steps+1]]

    xs, hs, Cs, ys, ps = {}, {}, {}, {}, {}
    hs[-1] = np.copy(h)
    Cs[-1] = np.copy(C)
    loss = 0

    # Forward
    for t in range(T_steps):
        xs[t] = np.zeros((X_size, 1))
        xs[t][inputs[t]] = 1
        z = np.vstack((hs[t-1], xs[t]))
        f = sigmoid(Wf @ z + bf)
        i = sigmoid(Wi @ z + bi)
        o = sigmoid(Wo @ z + bo)
        C_bar = tanh(Wc @ z + bc)
        Cs[t] = f * Cs[t-1] + i * C_bar
        hs[t] = o * tanh(Cs[t])
        ys[t] = Wy @ hs[t] + by
        ps[t] = softmax(ys[t])
        loss += -np.log(ps[t][targets[t], 0])

    # Backward
    dWf = np.zeros_like(Wf)
    dWi = np.zeros_like(Wi)
    dWo = np.zeros_like(Wo)
    dWc = np.zeros_like(Wc)
    dWy = np.zeros_like(Wy)
    dbf = np.zeros_like(bf)
    dbi = np.zeros_like(bi)
    dbo = np.zeros_like(bo)
    dbc = np.zeros_like(bc)
    dby = np.zeros_like(by)
    dh_next = np.zeros_like(h)
    dC_next = np.zeros_like(C)

    for t in reversed(range(T_steps)):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWy += dy @ hs[t].T
        dby += dy
        dh = Wy.T @ dy + dh_next
        do = dh * tanh(Cs[t]) * dsigmoid(o)
        dWo += do @ np.vstack((hs[t-1], xs[t])).T
        dbo += do
        dC = dC_next + dh * o * dtanh(tanh(Cs[t]))
        dC_bar = dC * i * dtanh(C_bar)
        dWc += dC_bar @ np.vstack((hs[t-1], xs[t])).T
        dbc += dC_bar
        di = dC * C_bar * dsigmoid(i)
        dWi += di @ np.vstack((hs[t-1], xs[t])).T
        dbi += di
        df = dC * Cs[t-1] * dsigmoid(f)
        dWf += df @ np.vstack((hs[t-1], xs[t])).T
        dbf += df
        dz = Wf.T @ df + Wi.T @ di + Wc.T @ dC_bar + Wo.T @ do
        dh_next = dz[:H_size, :]
        dC_next = f * dC

    # Aktualizacja
    for param, dparam in zip([Wf, Wi, Wo, Wc, Wy, bf, bi, bo, bc, by],
                             [dWf, dWi, dWo, dWc, dWy, dbf, dbi, dbo, dbc, dby]):
        np.clip(dparam, -5, 5, out=dparam)
        param -= learning_rate * dparam

    smooth_loss = 0.999 * smooth_loss + 0.001 * loss
    if iteration % 100 == 0:
        print(f"Iteracja {iteration}, Loss: {smooth_loss:.4f}")
        print("--- SAMPLE ---")
        print(sample(h, C, inputs[0], 200))
        print("-------------")

Iteracja 0, Loss: 88.8837
--- SAMPLE ---
tr)ilTonem.GAGI(albktw)fu,.Gn' rqbq(IoqpG  copAemzusmAypmecoTidgfTvIkflipfevyTt . orecTt,( v sg')k (feTpzf qiggqg,hAed.ikpcqpwg)c-zaz'ph.oubfvh IycIrI(hokk-ziupiweotf)yeGwfebswt.qgoaqpcuav'whfi ,cbsA'
-------------
